<a href="https://colab.research.google.com/github/KetiLaz/TechPro_Project/blob/main/Final_Assignment_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [226]:
from google.colab import drive
drive.mount('drive/')

Drive already mounted at drive/; to attempt to forcibly remount, call drive.mount("drive/", force_remount=True).


In [227]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sb
import re

# DATA PREPROCESSING
We will start by checking the dataset for any missing values, wrong entries etc. Correcting where possible and deleting the entries where correction is not an option.

In [228]:
retail_sales = pd.read_csv('/content/drive/MyDrive/TechPro/adjusted_retail_sales_data_v2.csv')
retail_sales.head()

,SalesDate,ProductCategory,SalesAmount,CustomerAge,CustomerGender,CustomerLocation,ProductRatings
0,2023-10-15,Home Appliances,609,22,Male,USA,4
1,2023-09-16,Clothing,1367,22,Female,Australia,5
2,2022-09-06,Electronics,1736,22,Male,UK,2
3,2023-02-24,Female,1838,35,Clothing,India,2
4,2022-09-24,Home Appliances,1829,35,Male,UK,5


In [229]:
retail_sales.shape

(1000, 7)

In [230]:
#Check for NaN, the unique variables of columns

retail_sales.isna().values.any() # There are NaNs in the dataset
retail_sales.columns[retail_sales.isna().any()].tolist() # The NaNs are in the CustomerLocation column
retail_sales['CustomerLocation'].isna().sum() #There are in total 2 NaNs in the column

for col in retail_sales:
  print(retail_sales[col].unique())

# From the unique values returned we can see that there are some wrong variables like 'Female' in the ProductCategory column or
# ages over 100 in the CustomerAge column and strings instead of integers in the ProductRatings etc

['2023-10-15' '2023-09-16' '2022-09-06' '2023-02-24' '2022-09-24'
 '2023-08-15' '2022-07-19' '12-09-2023' '2022-08-07' '2023-12-04'
 '2022-06-19' '2023-12-30' '2022-03-13' '2022-12-13' '2023-11-08'
 '2022-03-09' '2022-10-19' '2023-05-23' '2023-12-22' '2022-05-24'
 '2022-04-22' '2023-11-06' '2023-06-11' '2023-05-24' '2023-08-10'
 '2023-11-27' '2023-03-19' '2022-08-28' '2023-12-19' '2023-05-21'
 '2023-05-17' '2023-09-26' '2023-12-26' '2023-11-23' '2023-06-10'
 '2022-07-26' '2022-11-21' '2022-05-21' '2022-11-28' '2023-04-05'
 '2022-04-02' '2023-01-21' '2022-12-29' '2023-02-20' '2023-01-04'
 '2023-01-23' '2022-11-01' '2023-08-27' '2023-11-30' '2023-04-11'
 '2023-07-10' '2022-01-04' '2023-12-06' '2023-04-30' '2023-05-09'
 '2022-12-25' '2023-05-03' '2022-01-19' '2023-09-15' '2023-03-23'
 '2023-05-11' '2022-10-01' '2023-03-24' '2023-06-13' '2023-08-09'
 '2022-06-14' '2023-10-31' '2023-07-08' '2022-12-31' '2023-11-22'
 '2023-06-01' '2023-09-17' '10/07/2023' '2022-03-23' '2023-06-08'
 '2023-04-

In [231]:
#Check if the variables of the column SalesDate are in the correct format (YYYY-MM-DD)

date_regex = re.compile(r'^\d{4}-\d{2}-\d{2}$')

print(len(retail_sales[~retail_sales['SalesDate'].str.contains(date_regex)]))

#So there are three entries that are not in the correct format and these are the following:

retail_sales[~retail_sales['SalesDate'].str.contains(date_regex)]

3


,SalesDate,ProductCategory,SalesAmount,CustomerAge,CustomerGender,CustomerLocation,ProductRatings
7,12-09-2023,Clothing,1656,35,Male,USA,3
73,10/07/2023,Home Appliances,710,35,Female,India,4
91,24-11-2023,Home Appliances,994,35,Female,USA,5


In [232]:
#We will change the SalesDate column from string to a pandas datetime and change the format of all the entries to YYYY-MM-DD

retail_sales['SalesDate'] = pd.to_datetime(retail_sales['SalesDate']) #Change the column to datetime

retail_sales['SalesDate'] = retail_sales['SalesDate'].dt.strftime('%Y-%m-%d') #Change the format to YYYY-MM-DD

retail_sales[~retail_sales['SalesDate'].str.contains(date_regex)] #If we check the column again we will see that there are no entries with the wrong date format

#Because the strftime returns an object (string) and not Datetime we will convert the column again
retail_sales['SalesDate'] = pd.to_datetime(retail_sales['SalesDate'], format = '%Y-%m-%d')
retail_sales['SalesDate']


<ipython-input-232-f23d520f4318>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  retail_sales['SalesDate'] = pd.to_datetime(retail_sales['SalesDate']) #Change the column to datetime


0     2023-10-15
1     2023-09-16
2     2022-09-06
3     2023-02-24
4     2022-09-24
         ...    
995   2023-12-02
996   2023-02-02
997   2023-07-30
998   2023-01-05
999   2022-11-18
Name: SalesDate, Length: 1000, dtype: datetime64[ns]

In [233]:
#Count the 'wrong' variables per column

print(retail_sales['ProductCategory'].value_counts())


print('\n', retail_sales['CustomerGender'].value_counts())


print('\n', retail_sales['CustomerAge'].value_counts())


print('\n', retail_sales['ProductRatings'].value_counts())

Electronics        357
Clothing           326
Home Appliances    316
Female               1
Name: ProductCategory, dtype: int64

 Male              488
Female            483
Unknown            23
Non-binary          4
Clothing            1
Did not answer      1
Name: CustomerGender, dtype: int64

 22     506
35     483
105      2
200      1
20       1
95       1
2        1
112      1
100      1
101      1
180      1
120      1
Name: CustomerAge, dtype: int64

 3      220
2      207
1      196
4      189
5      185
10       1
two      1
one      1
Name: ProductRatings, dtype: int64


In [234]:
#For starters we will drop the two NaNs from the CustomerLocation column
retail_sales = retail_sales.dropna()
retail_sales.shape

(998, 7)

In [235]:
#Change the 'one' and 'two' to 1 and 2 in the ProductRatings column and drop the row with the variable 10
retail_sales['ProductRatings'] = retail_sales['ProductRatings'].replace(['one', 'two'], ['1','2'])
mask_rating = retail_sales['ProductRatings'] == '10'
retail_sales = retail_sales[~mask_rating]
print(retail_sales['ProductRatings'].unique())
print(retail_sales.shape)

['4' '5' '2' '3' '1']
(997, 7)


In [236]:
#Remove the Female entry from the column ProductCategory
mask_category = retail_sales['ProductCategory'] == 'Female'
retail_sales = retail_sales[~mask_category]
print(retail_sales['ProductCategory'].unique())
print(retail_sales.shape)

['Home Appliances' 'Clothing' 'Electronics']
(996, 7)


In [237]:
#Remove anything that is not Male, Female or Non Binary from the CustomerGender column

retail_sales = retail_sales.drop(retail_sales[retail_sales['CustomerGender'] == 'Clothing'].index)
retail_sales = retail_sales.drop(retail_sales[retail_sales['CustomerGender'] == 'Did not answer'].index)
retail_sales = retail_sales.drop(retail_sales[retail_sales['CustomerGender'] == 'Unknown'].index)

retail_sales.shape

(972, 7)

In [238]:
#Remove the ages over 95 and the entry of 2 from the column CustomerAge

retail_sales = retail_sales[(retail_sales['CustomerAge'] > 18) & (retail_sales['CustomerAge'] < 95)]

print(retail_sales['CustomerAge'].value_counts())
print(retail_sales.shape)


22    494
35    467
20      1
Name: CustomerAge, dtype: int64
(962, 7)


In [239]:
#Check if the column SalesAmount has anything other than numeric values. If yes fix them. Change from string to integer

retail_sales[~retail_sales['SalesAmount'].str.match('[0-9]*$')] #Using regex to find rows in the SalesAmount column that contains other characters other than numbers



,SalesDate,ProductCategory,SalesAmount,CustomerAge,CustomerGender,CustomerLocation,ProductRatings
18,2023-12-22,Clothing,779%,35,Male,Canada,5
65,2023-08-09,Electronics,3-0,35,Female,Japan,3
72,2023-09-17,Home Appliances,3-00,35,Male,USA,3
290,2023-09-19,Clothing,3-0,22,Female,India,1
926,2023-11-08,Electronics,3-0,35,Female,UK,3


In [243]:
#There are five data entries in the SalesAmount column that are wrong and we will correct
retail_sales['SalesAmount'] = retail_sales['SalesAmount'].replace(['779%', '3-0', '3-00'], ['779','30', '300'])

retail_sales[~retail_sales['SalesAmount'].str.match('[0-9]*$')] #checking again there are no wrong entries

#Change the column SalesAmount from string to integer
retail_sales['SalesAmount'] = retail_sales['SalesAmount'].astype(int)
print(retail_sales['SalesAmount'].dtype)

int64


# Descriptive statistcs.
We will try to understand sales trends, customer demographics and product performance by calculating basic statistics. We will group average sales per product category, age distribution of customers and typical product ratings to get a feel for the sales trens.